In [1]:
# Import libraries
import requests

/Users/winsonchow/Documents/GitHub/RAG-app/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Define API endpoints
list_posts_url = 'https://www.techinasia.com/wp-json/techinasia/2.0/posts'
search_posts_url = 'https://www.techinasia.com/wp-json/techinasia/2.0/articles?query='

In [3]:
# Function to fetch posts from API
def fetch_posts_from_api(page=1):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(list_posts_url, params={'page': page}, headers=headers)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        print("Response Headers:", response.headers)
        print("Response Text:", response.text)

        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f'Error fetching posts: {e}')
        return None
    except ValueError as e:
        print(f'Error parsing response: {e}')
        return None

# Function to load posts from JSON file
def load_posts_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError as e:
        print(f'Error loading posts from file: {e}')

# Main function to get posts
def get_posts(page=1, file_path='/data/tia_page_1.json'):
    posts_data = fetch_posts_from_api(page=page)
    if posts_data and 'posts' in posts_data:
        posts = posts_data['posts']
    else:
        print('Falling back to the JSON file.')
        posts = load_posts_from_file(file_path)

    return posts

In [4]:
# Function to search posts
def search_posts(query):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(search_posts_url, params={'query': query}, headers=headers)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        print("Response Headers:", response.headers)
        print("Response Text:", response.text)

        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f'Error fetching posts: {e}')
        return None
    except ValueError as e:
        print(f'Error parsing response: {e}')
        return None

In [5]:
# Fetch posts
posts = get_posts(page=2)
print(f"Fetched {len(posts)} posts.")

# Display the first post to verify the structure
if posts:
    print(posts[0])

Status Code: 200
Response Headers: {'Access-Control-Allow-Headers': 'Authorization, X-WP-Nonce, Content-Disposition, Content-MD5, Content-Type', 'Access-Control-Allow-Origin': 'www.channelnewsasia.com', 'Allow': 'GET', 'Cache-Control': 'no-cache, no-store, must-revalidate, no-transform', 'Content-Encoding': 'gzip', 'Content-Security-Policy': "script-src 'unsafe-inline' 'unsafe-eval' https:; worker-src blob:;", 'Content-Type': 'application/json; charset=UTF-8', 'Date': 'Fri, 24 May 2024 09:12:17 GMT', 'Expires': '0', 'Front-End-Https': 'on', 'Pragma': 'no-cache', 'Server': 'nginx', 'Strict-Transport-Security': 'max-age=31536000;', 'Vary': 'Accept-Encoding, Origin', 'X-Content-Type-Options': 'nosniff, nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'X-Powered-By': 'PHP/7.2.6', 'X-Robots-Tag': 'noindex', 'X-UA-Compatible': 'IE=Edge', 'X-XSS-Protection': '1; mode=block', 'transfer-encoding': 'chunked', 'Connection': 'keep-alive'}
Response Text: {"total":52955,"per_page":30,"current_page":2,"tot

In [6]:
# # Search for posts with a keyword
# keyword = "What are the upcoming Events at Tech in Asia?"

# search_results = search_posts(keyword)

# # if search_results:
# #     if 'posts' in search_results and 'hits' in search_results['posts']:
# #         search_posts = search_results['posts']['hits']
# #     else:
# #         search_posts = []

# #     print(f"Found {len(search_posts)} posts with keyword '{keyword}'.")
# #     # Display the first search result to verify the structure
# #     if search_posts:
# #         print(search_posts[0])
# #         print("\n")
# #         print(search_posts[0]['title'])
# #         print("\n")
# #         print(search_posts[0]['content'])


# # search_results = search_posts(query)

# if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
#     search_posts = search_results['posts']['hits']
#     # context with title only
#     context = " ".join([post['title'] for post in search_posts[:10]])

#     # context with title and content
#     # context = "".join([post['title'] + post['content'] for post in search_posts[:3]])

#     print(f"Context: {context}")
# else:
#     context = 'No relevant posts found. Please provide more context.'

In [14]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import gradio as gr

In [15]:
load_dotenv()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [16]:
# Function to generate responses using OpenAI
def generate_response(search_results, query):
    try:
        if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
            search_posts = search_results['posts']['hits']
            # context with title only
            context = " ".join([post['title'] for post in search_posts[:2]])

            # context with title and content
            # context = "".join([post['title'] + post['content'] for post in search_posts[:3]])
        else:
            context = 'No relevant posts found. Please provide more context.'

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Question: {query}\nContext: {context}\nAnswer:"}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content
    
    except Exception as e:
        print(f'Error generating response: {e}')
        return None

In [17]:
query = 'What are the upcoming Events at Tech in Asia?'

search_results = search_posts(query)
generate_response(search_results, query)

Status Code: 200
Response Headers: {'Access-Control-Allow-Headers': 'Authorization, X-WP-Nonce, Content-Disposition, Content-MD5, Content-Type', 'Access-Control-Allow-Origin': 'www.channelnewsasia.com', 'Allow': 'GET', 'Cache-Control': 'no-cache, no-store, must-revalidate, no-transform', 'Content-Encoding': 'gzip', 'Content-Security-Policy': "script-src 'unsafe-inline' 'unsafe-eval' https:; worker-src blob:;", 'Content-Type': 'application/json; charset=UTF-8', 'Date': 'Fri, 24 May 2024 09:14:41 GMT', 'Expires': '0', 'Front-End-Https': 'on', 'Pragma': 'no-cache', 'Server': 'nginx', 'Strict-Transport-Security': 'max-age=31536000;', 'Vary': 'Accept-Encoding, Origin', 'X-Content-Type-Options': 'nosniff, nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'X-Powered-By': 'PHP/7.2.6', 'X-Robots-Tag': 'noindex', 'X-UA-Compatible': 'IE=Edge', 'X-XSS-Protection': '1; mode=block', 'transfer-encoding': 'chunked', 'Connection': 'keep-alive'}
Response Text: {"posts":{"hits":[{"title":"Ex-Tokocrypto CEO want

"I'm sorry, but I do not have up-to-date information on specific upcoming events at Tech in Asia. I recommend checking their official website or social media channels for the latest updates on their events calendar."